# Strategy Analysis Example

Execute the single indicator template via Runner.submit through the WorldService and plot the results.

In [ ]:
import polars as pl
import matplotlib.pyplot as plt

from qmtl.examples.templates.single_indicator import SingleIndicatorStrategy
from qmtl.runtime.sdk import Runner

strategy = Runner.submit(
    SingleIndicatorStrategy,
    world_id='single_indicator_example',
    gateway_url='http://localhost:8000'
)

price_snap = strategy.nodes[0].cache._snapshot()[strategy.nodes[0].node_id][strategy.nodes[0].interval]
ema_snap = strategy.nodes[1].cache._snapshot()[strategy.nodes[1].node_id][strategy.nodes[1].interval]

price = pl.DataFrame(price_snap, schema=['ts', 'payload'])
price = price.with_columns(
    pl.col('payload').map_elements(lambda p: p['close']).alias('close')
)
ema = pl.DataFrame(ema_snap, schema=['ts', 'ema'])

merged = price.select(['ts', 'close']).join(ema, on='ts', how='inner')

ts = merged.get_column('ts').to_list()
close = merged.get_column('close').to_list()
ema_values = merged.get_column('ema').to_list()

plt.plot(ts, close, label='close')
plt.plot(ts, ema_values, label='ema')
plt.legend()
plt.show()
